In [1]:
import torch
torch.set_printoptions(precision=2, sci_mode=False)
import numpy as np


In [2]:
def load_tensor(filename):
    metadata = {}
    data_values = []
    
    # List of metadata keys to look for
    meta_keys = ['size', 'ndim', 'shape', 'stride', 'elem_size', 'requires_grad']

    with open(filename, 'r') as f:
        for line in f:
            # Clean up line: remove spaces, split by comma
            parts = [p.strip() for p in line.split(',') if p.strip()]
            
            if not parts:
                continue  # Skip empty lines
            
            # Check if this row starts with a metadata label
            label = parts[0]
            
            if label in meta_keys:
                # If there are numbers after the label, store them
                vals = [int(v) for v in parts[1:]]
                metadata[label] = vals[0] if len(vals) == 1 else vals
            else:
                # If it's not a label, check if the parts are numbers (the tensor data)
                try:
                    row_data = [float(p) for p in parts]
                    data_values.extend(row_data)
                except ValueError:
                    # Not a number and not a metadata key (could be a header or text)
                    continue

    # Convert to PyTorch Tensor
    if 'shape' in metadata:
        # Convert floats to ints for reshaping (e.g., 2.0 -> 2)
        target_shape = [int(s) for s in metadata['shape']]
        tensor = torch.tensor(data_values).reshape(target_shape)
        return metadata, tensor
    
    return metadata, torch.tensor(data_values)

In [3]:
meta_tensor1, tensor1 = load_tensor('../output/tensor1.csv')
meta_tensor2, tensor2 = load_tensor('../output/tensor2.csv')
meta_output_tensor, output_tensor_c = load_tensor('../output/output_tensor.csv')

In [4]:
print("Metadata:", meta_tensor1)
display(tensor1)
print("Metadata:", meta_tensor2)
display(tensor2)
print("Metadata:", meta_output_tensor)
display(output_tensor_c)

Metadata: {'size': 24, 'ndim': 3, 'shape': [2, 4, 3], 'stride': [12, 3, 1], 'elem_size': 8, 'requires_grad': 0}


tensor([[[-10.00,  -7.37,   5.11],
         [ -0.83,   0.66,  -5.62],
         [ -9.06,   3.58,   3.59],
         [  8.69,  -2.33,   0.39]],

        [[  6.62,  -9.31,  -8.93],
         [  0.59,   3.42,  -9.85],
         [ -2.33,  -8.66,  -1.65],
         [  3.74,   1.78,   8.61]]])

Metadata: {'size': 12, 'ndim': 2, 'shape': [3, 4], 'stride': [4, 1], 'elem_size': 8, 'requires_grad': 0}


tensor([[ 6.92,  0.54, -8.16,  3.08],
        [-1.68,  4.02,  8.21,  5.24],
        [-4.75, -9.05,  4.72, -3.44]])

Metadata: {'size': 32, 'ndim': 3, 'shape': [2, 4, 4], 'stride': [16, 4, 1], 'elem_size': 8, 'requires_grad': 0}


tensor([[[ -81.14,  -81.31,   45.27,  -86.99],
         [  19.88,   53.06,  -14.41,   20.20],
         [ -85.77,  -22.94,  120.22,  -21.45],
         [  62.26,   -8.21,  -88.24,   13.21]],

        [[ 103.90,   46.94, -172.58,    2.24],
         [  45.14,  103.21,  -23.25,   53.60],
         [   6.25,  -21.18,  -59.86,  -46.94],
         [ -18.03,  -68.74,   24.77,   -8.74]]])

In [5]:
tensor1.shape

torch.Size([2, 4, 3])

In [6]:
tensor2.shape

torch.Size([3, 4])

In [7]:
output_tensor_py = tensor1 @ tensor2
output_tensor_py.shape

torch.Size([2, 4, 4])

In [8]:
output_tensor_py

tensor([[[ -81.09,  -81.27,   45.21,  -87.00],
         [  19.84,   53.07,  -14.33,   20.23],
         [ -85.76,  -22.99,  120.27,  -21.50],
         [  62.20,   -8.20,  -88.20,   13.21]],

        [[ 103.87,   46.97, -172.60,    2.32],
         [  45.12,  103.21,  -23.23,   53.62],
         [   6.26,  -21.14,  -59.87,  -46.88],
         [ -18.01,  -68.75,   24.73,   -8.77]]])

In [9]:
output_tensor_c

tensor([[[ -81.14,  -81.31,   45.27,  -86.99],
         [  19.88,   53.06,  -14.41,   20.20],
         [ -85.77,  -22.94,  120.22,  -21.45],
         [  62.26,   -8.21,  -88.24,   13.21]],

        [[ 103.90,   46.94, -172.58,    2.24],
         [  45.14,  103.21,  -23.25,   53.60],
         [   6.25,  -21.18,  -59.86,  -46.94],
         [ -18.03,  -68.74,   24.77,   -8.74]]])

In [10]:
output_tensor_py - output_tensor_c

tensor([[[ 0.05,  0.04, -0.06, -0.01],
         [-0.04,  0.01,  0.08,  0.03],
         [ 0.01, -0.05,  0.05, -0.05],
         [-0.06,  0.01,  0.04,  0.00]],

        [[-0.03,  0.03, -0.02,  0.08],
         [-0.02, -0.00,  0.02,  0.02],
         [ 0.01,  0.04, -0.01,  0.06],
         [ 0.02, -0.01, -0.04, -0.03]]])

In [60]:
output_tensor_py.shape

torch.Size([2, 4, 4])

In [11]:
output_tensor_c.shape

torch.Size([2, 4, 4])